In [ ]:
import pendulum
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

local_tz = pendulum.timezone("Asia/Bangkok")

name='TraceLocation'
prefix='DMS_'
csv_path = '/usr/local/airflow/plugins'+'/'

dag_params = {
    'owner': 'airflow',
    "depends_on_past": False,
    'start_date': datetime(2022, 4, 4, tzinfo=local_tz),
    'email_on_failure': True,
    'email_on_retry': False,
    'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
    'do_xcom_push': False,
    'execution_timeout':timedelta(seconds=300)
    # 'retries': 3,
    # 'retry_delay': timedelta(minutes=10),
}

dag = DAG(prefix+name,
          catchup=False,
          default_args=dag_params,
          schedule_interval= '5 * * * *',
          tags=[prefix+name, 'Daily', '60mins']
)

In [3]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [1]:
from utils.df_handle import *


datenow = datetime.now().strftime("%Y-%m-%d")
datenow_mns1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
fdom = datetime.now().replace(day=1).strftime("%Y%m%d")

# datenow_mns1 = '20220331'
# datenow_mns2 = '20220403'
sql = \
f"""
WITH T1 as
(
SELECT
BranchID,
SlsperID,
Lat,
Lng,
CustID,
Type as Typ,
NumberCICO,
UpdateTime as CheckInUpdateTime
from dbo.AR_SalespersonLocationTrace 
where CAST (UpdateTime as DATE) >= '{datenow_mns1}'
and CAST (UpdateTime as DATE) <= '{datenow_mns1}'
and Type = 'IO'
--ORDER BY UpdateTime ASC
)

--select top 100 SUBSTRING(Type,1,2), Type from dbo.AR_SalespersonLocationTrace where CAST (UpdateTime as DATE) = '2022-03-25'
, T2 as
(
SELECT
BranchID,
SlsperID,
SUBSTRING(Type,3,20) as DeOrderNbr,
UpdateTime as DE_UpdateTime,
NumberCICO
from
dbo.AR_SalespersonLocationTrace
where CAST (UpdateTime as DATE) >= '{datenow_mns1}'
and CAST (UpdateTime as DATE) <= '{datenow_mns1}'
and SUBSTRING(Type,1,2) = 'DE'
)

Select
T1.BranchID,
T1.SlsperID,
Lat,
Lng,
CustID,
Typ,
T1.NumberCICO,
CheckInUpdateTime,
DeOrderNbr,
DE_UpdateTime
from T1
LEFT JOIN T2 ON
T1.NumberCICO = T2.NumberCICO
and T1.BranchID = T2.BranchID
and T1.SlsperID = T1.SlsperID
ORDER BY CheckInUpdateTime ASC
"""

def insert():
    TRACE = get_ms_df(sql)
    TRACE.DeOrderNbr = np.where(TRACE.DeOrderNbr.isna(), None, TRACE.DeOrderNbr)
    TRACE.DE_UpdateTime = pd.to_datetime(TRACE.DE_UpdateTime, dayfirst=True)
    TRACE.DE_UpdateTime.fillna(datetime(1900,1,1), inplace=True)
    execute_values_insert(TRACE, "d_trace_location")

Data inserted using execute_values() successfully..


In [4]:
TRACE = get_ms_df(sql)

In [5]:
TRACE.head()

,BranchID,SlsperID,Lat,Lng,CustID,Typ,NumberCICO,UpdateTime
0,MR0001,MR1360,10.791951,106.776781,M0901033,IO,bdb503f4-b3d7-4811-8f2b-ccb8120f051c,2022-05-04 00:01:17
1,MR0001,MR1360,10.791988,106.776806,M0202042,IO,aa157ab7-c5f3-413c-bb98-72c3d190db83,2022-05-04 00:02:28
2,MR0010,MR2535,20.989818,105.818711,HH11O1113,IO,c1a8e721-28fd-4152-99c9-e4a56358ab41,2022-05-04 00:13:42
3,MR0010,MR2532,20.957986,107.122893,TD32O733,IO,5a392468-b526-4ff2-9afb-7532b6cf9893,2022-05-04 00:55:26
4,MR0010,MR2532,20.957992,107.122895,TD32E014,IO,5000e0b4-e127-40e5-af0a-6fafb0581dc7,2022-05-04 00:56:36


In [6]:
datenow = datetime.now().strftime("%Y-%m-%d")
datenow_mns1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
fdom = datetime.now().replace(day=1).strftime("%Y%m%d")

In [1]:
sql = \
f"""
DECLARE @from DATE = '{datenow_mns1}'
DECLARE @to DATE = '{datenow_mns1}'

SELECT
BranchID,
SlsperID,
Lat,
Lng,
CustID,
Type as Typ,
CheckType = 'IO',
NumberCICO,
UpdateTime
from dbo.AR_SalespersonLocationTrace 
where CAST (UpdateTime as DATE) >= @from
and CAST (UpdateTime as DATE) <= @to
and [Type] = 'IO'

UNION ALL

SELECT
BranchID,
SlsperID,
Lat,
Lng,
CustID,
Type as Typ,
CheckType = 'OO',
NumberCICO,
UpdateTime
from dbo.AR_SalespersonLocationTrace 
where CAST (UpdateTime as DATE) >= @from
and CAST (UpdateTime as DATE) <= @to
and [Type] = 'OO'

UNION ALL

SELECT
BranchID,
SlsperID,
Lat,
Lng,
CustID,
Type,
CheckType = 'MaxAction',
a.NumberCICO,
UpdateTime
from dbo.AR_SalespersonLocationTrace a
INNER JOIN
(
SELECT
NumberCICO,
max(UpdateTime) as MaxUpdateTime
from dbo.AR_SalespersonLocationTrace a
where CAST (UpdateTime as DATE) >= @from
and CAST (UpdateTime as DATE) <= @to
and [Type] not in ('OO','IO')
group by NumberCICO
) b
ON a.NumberCICO = b.NumberCICO
and a.UpdateTime = MaxUpdateTime
where CAST (UpdateTime as DATE) >= @from
and CAST (UpdateTime as DATE) <= @to
and [Type] not in ('OO','IO')
"""
TRACE = get_ms_df(sql)

try:
    assert TRACE.shape[0] > 0
    bq_values_insert(TRACE, "d_checkin", 2)
except AssertionError:
    print("There is no data")

In [9]:
# DROP TABLE IF EXISTS biteam.d_checkin;
# CREATE TABLE biteam.d_checkin LIKE biteam.d_checkin_temp
# PARTITION BY DATE(updatetime)
# CLUSTER BY custid,typ

In [ ]:
dummy_start = DummyOperator(task_id="dummy_start", dag=dag)

insert = PythonOperator(task_id="insert", python_callable=insert, dag=dag)

dummy_start >> insert